In [1]:
import pymongo

from pymongo import MongoClient
client = MongoClient('localhost',27017)
#now access the desired db
db = client.test
testData = db.testData

In [6]:
#let's insert a record into the db.  Records are stored as python dictionaries
import datetime
blogPost = {"author":"Peter",
            "title":"The amazing Peter Dude",
            "tags":["mongodb","stuperheroes","examples"],
            "date":datetime.datetime.utcnow()
    }

posts = db.posts # note that if this collection doesn't yet exist, mongo will create it
posts_id = posts.insert(blogPost) #note that the documentation says to use insert_one.  I found that insert_one doesn't work, but that insert does.


In [3]:
posts.find_one() #note that if there are multiple documents matching this query, it'll only return the first

{u'_id': ObjectId('5509ad48c660b10347423e4d'),
 u'author': u'Larry',
 u'date': datetime.datetime(2015, 3, 18, 16, 52, 24, 762000),
 u'tags': [u'mongodb', u'superheroes', u'examples'],
 u'title': u'The amazing spiderman'}

In [7]:
#a few examples of more advanced queries

#searching for an exact match on a field
posts.find_one({'author':'Peter'})

{u'_id': ObjectId('55428d5e62bc37068480575e'),
 u'author': u'Peter',
 u'date': datetime.datetime(2015, 4, 30, 20, 15, 26, 174000),
 u'tags': [u'mongodb', u'stuperheroes', u'examples'],
 u'title': u'The amazing Peter Dude'}

In [5]:
#returning multiple results
for post in posts.find(): #note that find creates a cursor object, whereas find_one returns a single document
    print post

{u'date': datetime.datetime(2015, 3, 18, 16, 52, 24, 762000), u'_id': ObjectId('5509ad48c660b10347423e4d'), u'author': u'Schuck', u'tags': [u'mongodb', u'superheroes', u'examples'], u'title': u'The amazing spiderman'}
{u'date': datetime.datetime(2015, 3, 18, 16, 52, 32, 582000), u'_id': ObjectId('5509ad50c660b10347423e4e'), u'author': u'Schuck', u'tags': [u'mongodb', u'superheroes', u'examples'], u'title': u'The amazing spiderman'}
{u'date': datetime.datetime(2015, 3, 18, 16, 52, 40, 455000), u'_id': ObjectId('5509ad58c660b10347423e4f'), u'author': u'Schuck', u'tags': [u'mongodb', u'superheroes', u'examples'], u'title': u'The amazing spiderman'}
{u'date': datetime.datetime(2015, 3, 18, 16, 54, 41, 147000), u'_id': ObjectId('5509add1c660b10347423e50'), u'author': u'John', u'tags': [u'mongodb', u'stuperheroes', u'examples'], u'title': u'The amazing JohnMan'}
{u'date': datetime.datetime(2015, 3, 18, 19, 34, 37, 582000), u'_id': ObjectId('5509d34dc660b10347423e51'), u'author': u'John', u't

In [6]:
#exclude the _id from the returned results we need to include info in the projections parameter
for post in posts.find({'author':'Peter'},{'_id':0}):
    print post

In [5]:
#update a record
db.posts.update({'author':"Peter"},{'$set':{'author':"Larry"}},upsert=False, multi=True)

{u'n': 0, u'nModified': 0, u'ok': 1, 'updatedExisting': False}

In [8]:
#to use a regular expression in pymongo as the search parameter takes a bit more work than in the mongo shell.  
# There are a few ways of accomplishing this.  First, let's look at the 'normal' js way

for post in db.posts.find({'tags':{'$regex':'^stup'}}):
    print post, '\n\r'

{u'date': datetime.datetime(2015, 3, 18, 16, 54, 41, 147000), u'_id': ObjectId('5509add1c660b10347423e50'), u'author': u'John', u'tags': [u'mongodb', u'stuperheroes', u'examples'], u'title': u'The amazing JohnMan'} 

{u'date': datetime.datetime(2015, 3, 18, 19, 34, 37, 582000), u'_id': ObjectId('5509d34dc660b10347423e51'), u'author': u'John', u'tags': [u'mongodb', u'stuperheroes', u'examples'], u'title': u'The amazing JohnMan'} 

{u'date': datetime.datetime(2015, 3, 19, 15, 23, 9, 357000), u'_id': ObjectId('550ae9dda00ac020156528e0'), u'author': u'John', u'tags': [u'mongodb', u'stuperheroes', u'examples'], u'title': u'The amazing JohnMan'} 



In [9]:
# and now the python way

import re #python's regex library
regx = re.compile("^P",re.IGNORECASE)
for post in db.posts.find({'author':regx}):
    print post

In [14]:
#let's delete a record using the remove() method
db.post({'author':'John'},{'justOne': True})

TypeError: 'Collection' object is not callable. If you meant to call the 'post' method on a 'Database' object it is failing because no such method exists.